# Counting distinct values

It's extremely common for analysts to want to count unique occurences of some dimension value in data. With the Druid database's history of large volumes of data comes an advanced computer science technique to speed up this calculation through approximation. In this tutorial, work through some examples and see the effect of turning it on and off, and of making it even faster by pre-generating the objects that Druid uses to execute the query.

## Prerequisites

This tutorial works with Druid 26.0.0 or later.

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [Docker for Jupyter Notebook tutorials](https://druid.apache.org/docs/latest/tutorials/tutorial-jupyter-docker.html).

<details><summary>    
<b>Run without Docker Compose</b>    
</summary>

If you do not use the Docker Compose environment, you need the following:

* A running Druid instance.
* [druidapi](https://github.com/apache/druid/blob/master/examples/quickstart/jupyter-notebooks/druidapi/README.md), a Python client for Apache Druid. Follow the instructions in the Install section of the README file.
* [matplotlib](https://matplotlib.org/), a library for creating visualizations in Python,
* [pandas](https://pandas.pydata.org/), a data analysis and manipulation tool.
* Jupyter notebook or Jupyter Lab. See [jupyter.org](https://jupyter.org/) for installation instructions.

</details>

### Initialization

Run the next cell to attempt a connection to Druid services. If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

### Load example flight data

Once your Druid environment is up and running, ingest the sample data for this tutorial.

Open the Druid console:

1. Load data
2. Batch - SQL
3. Example data
4. Select "FlightCarrierOnTime (1 month)"

For the purposes of this notebook, use all the defaults suggested by the console, including the default datasource name: 

`On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11`

When this is completed, run the following cell for the final part of the initialization. This will provide us some methods to call.

In [ ]:
import json
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

## COUNT(DISTINCT) queries on basic datasets

This is a typical query you might run to find the number of distinct Tail Numbers in the example dataset.

```sql
SELECT
    "Reporting_Airline",
    COUNT(DISTINCT "Tail_Number") AS "Events"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2
```

You're about to discover several ways enables you to arrive at this answer, each one having different performance characteristics depending on the needs of your users.

### Running COUNT(DISTINCT) with approximation

Druid automatically looks for query patterns that benefit from from approximation. In this instance, Druid identifies a match for approximate `COUNT(DISTINCT)`.

In Druid, each data server computes its own intermediate results that are then merged into the final result set that we see. In a `COUNT(DISTINCT)` query, they must create their own set of unique values, pass these back, and then computation is done to find uniques inside those larger sets. When the dimension being looked at contains many tens-of-thousands, perhaps even millions of unique values, both the transfer of intermediate results and the final computation can be both memory and CPU hungry.

When Druid uses approximation, the intermediate results are contained in a representation called a [data sketch](https://datasketches.apache.org/). These much smaller objects are transfered and merged together when the query results are finalized, rather than Druid having to combine the individual lists of distinct values from each process. The most scalable part of Druid – the individual data servers – do much more of the work, and do so earlier, instead of leaving it to the merge stage.

Interestingly, data sketch size is not dependent on the underlying data – the default size of a sketch in Druid is just over 2000 bytes.

This translates into much faster query execution, especially when the intermediate results are large.

> Approximations improve scalability, storage, and memory use - at the cost of some error.
> 
> _[Gian Merlino](https://github.com/gianm)_

In your own deployment, check Druid's [configuration files](https://druid.apache.org/docs/26.0.0/configuration/index.html#sql) to see what type of data sketch is being used (`druid.sql.approxCountDistinct.function`) and whether this approach has been left as the default by your system administrators (`druid.sql.planner.useApproximateCountDistinct`).

Run the following cell to plot the number of events in the sample data for each `Tail_Number`. Remember by default, each Druid process will be calculating its local results as a data sketch, passing these back for the final merge operation.

In [ ]:
sql = '''
SELECT
    "Reporting_Airline",
    COUNT(DISTINCT "Tail_Number") AS "Unique Tail Numbers"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2
'''

df1 = pd.DataFrame(sql_client.sql(sql))

df1.plot.bar(x='Reporting_Airline', y='Unique Tail Numbers')
plt.show()

### Running COUNT(DISTINCT) without approximation

We can supply a query context parameter, `useApproximateCountDistinct`, to force Druid to not use this approximation technique.

Using the same SQL statement as before, the following cell crafts a request (`req`) where we set the context parameter before Druid executes the query.

In [ ]:
req = sql_client.sql_request(sql)
req.add_context("useApproximateCountDistinct", "false")
resp = sql_client.sql_query(req)

df2 = pd.DataFrame(resp.rows)
df2.plot.bar(x='Reporting_Airline', y='Unique Tail Numbers')
plt.show()

### Comparing approximate and non-approximate results

The next cell shows a comparison of the two results above: `df1` used the default approximation approach, while `df2` are the results where we turned approximation off.

In [ ]:
df3 = df1.compare(df2, keep_equal=True)
df3

The table shows:

* A row number
* The reporting airline in the approximate results (`self`) versus that in the non-approximate results (`other`)
* The calculated distinct number of `Tail Number`s

Notice that there are _value_ errors, as you might expect with approximation, and that in some instances this affects the _order_ of results.

Error in sketch-based approximation is probabilistic, rather than guaranteed. That's to say that a certain percentage of the time you can expect the measurements you take to be within a certain distance of the true value.

As an experiment, you may want to:

* Identify a high-cardinality column in one of your own data sets
* Test how long an approximate `DISTINCT(COUNT)` query takes to run with approximation turned on
* Test how long the same query takes to run with approximation turned off

## COUNT(DISTINCT) queries on sketched data

For even faster performance, create a `TABLE` in Druid where sketches are stored directly in the table. At ingestion time, you create sets of unique values from the raw data that can then be addressed with SQL functions, including ones that solve for `COUNT(DISTINCT)` by addressing the sketches directly.

Take a look at the SQL in the next cell: it applies a `GROUP BY` to `Reporting_Airline` and uses the [`DS_HLL`](https://druid.apache.org/docs/26.0.0/querying/sql-functions.html#ds_hll) function to create sets of `Tail_Number`s inside a "HyperLogLog" sketch object.

Run the cell to get a "human readable" version of the result:

In [ ]:
sql = '''
SELECT
    "Reporting_Airline",
    DS_HLL("Tail_Number") AS "Sketch"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
LIMIT 5
'''

display.sql(sql)

The results show, for each `Reporting_Airline`, the highly optimized, aggregated list of the `Tail_Number`s. By storing this in the table, data processes gain an overall speed boost from both an increase in table data efficiency and a decrease in computation effort.

### Creating sketches during batch ingestion

The next cell ingests the example flight data into a new table, `flights-counts`. A `GROUP BY` aggregates all `Tail_Number`s into a HLL sketch using `DS_HLL` for each hour (`TIME_FLOOR`), `Reporting_Airline`, `Origin`, and `Dest`.

Notice that it does not store the original `Tail_Number`s. If we kept that field, the `GROUP BY` wouldn't aggregate any rows into the sketch - there would be a 1:1 relationship between the row and each `Tail_Number` - which is the opposite of what we are designing for! By implication, it will be no longer possible to use the raw data as part of any SQL queries, like a `GROUP BY` or a `WHERE`.

These two techniques – aggregation and generating sketches – increases query execution efficiency dramatically.

"HyperLogLog" is not the only type of sketch that can be used: there are two types of Apache Datasketch that allow for `COUNT(DISTINCT)` computations.

* [HyperLogLog](https://druid.apache.org/docs/26.0.0/querying/sql-aggregations.html#hll-sketch-functions)
* [Theta](https://druid.apache.org/docs/26.0.0/querying/sql-aggregations.html#theta-sketch-functions)

When you need to also carry out set operations, like intersection and difference, create Theta sketches using the [`DS_THETA`](https://druid.apache.org/docs/26.0.0/querying/sql-functions.html#ds_theta) function. You can see this happening in the SQL in the cell below.

In [ ]:
sql='''
REPLACE INTO "flights-counts" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://static.imply.io/example-data/flight_on_time/flights/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11.csv.zip"]}',
    '{"type":"csv","findColumnsFromHeader":true}'
  )
) EXTEND ("depaturetime" VARCHAR, "arrivalime" VARCHAR, "Year" BIGINT, "Quarter" BIGINT, "Month" BIGINT, "DayofMonth" BIGINT, "DayOfWeek" BIGINT, "FlightDate" VARCHAR, "Reporting_Airline" VARCHAR, "DOT_ID_Reporting_Airline" BIGINT, "IATA_CODE_Reporting_Airline" VARCHAR, "Tail_Number" VARCHAR, "Flight_Number_Reporting_Airline" BIGINT, "OriginAirportID" BIGINT, "OriginAirportSeqID" BIGINT, "OriginCityMarketID" BIGINT, "Origin" VARCHAR, "OriginCityName" VARCHAR, "OriginState" VARCHAR, "OriginStateFips" BIGINT, "OriginStateName" VARCHAR, "OriginWac" BIGINT, "DestAirportID" BIGINT, "DestAirportSeqID" BIGINT, "DestCityMarketID" BIGINT, "Dest" VARCHAR, "DestCityName" VARCHAR, "DestState" VARCHAR, "DestStateFips" BIGINT, "DestStateName" VARCHAR, "DestWac" BIGINT, "CRSDepTime" BIGINT, "DepTime" BIGINT, "DepDelay" BIGINT, "DepDelayMinutes" BIGINT, "DepDel15" BIGINT, "DepartureDelayGroups" BIGINT, "DepTimeBlk" VARCHAR, "TaxiOut" BIGINT, "WheelsOff" BIGINT, "WheelsOn" BIGINT, "TaxiIn" BIGINT, "CRSArrTime" BIGINT, "ArrTime" BIGINT, "ArrDelay" BIGINT, "ArrDelayMinutes" BIGINT, "ArrDel15" BIGINT, "ArrivalDelayGroups" BIGINT, "ArrTimeBlk" VARCHAR, "Cancelled" BIGINT, "CancellationCode" VARCHAR, "Diverted" BIGINT, "CRSElapsedTime" BIGINT, "ActualElapsedTime" BIGINT, "AirTime" BIGINT, "Flights" BIGINT, "Distance" BIGINT, "DistanceGroup" BIGINT, "CarrierDelay" BIGINT, "WeatherDelay" BIGINT, "NASDelay" BIGINT, "SecurityDelay" BIGINT, "LateAircraftDelay" BIGINT, "FirstDepTime" VARCHAR, "TotalAddGTime" VARCHAR, "LongestAddGTime" VARCHAR, "DivAirportLandings" VARCHAR, "DivReachedDest" VARCHAR, "DivActualElapsedTime" VARCHAR, "DivArrDelay" VARCHAR, "DivDistance" VARCHAR, "Div1Airport" VARCHAR, "Div1AirportID" VARCHAR, "Div1AirportSeqID" VARCHAR, "Div1WheelsOn" VARCHAR, "Div1TotalGTime" VARCHAR, "Div1LongestGTime" VARCHAR, "Div1WheelsOff" VARCHAR, "Div1TailNum" VARCHAR, "Div2Airport" VARCHAR, "Div2AirportID" VARCHAR, "Div2AirportSeqID" VARCHAR, "Div2WheelsOn" VARCHAR, "Div2TotalGTime" VARCHAR, "Div2LongestGTime" VARCHAR, "Div2WheelsOff" VARCHAR, "Div2TailNum" VARCHAR, "Div3Airport" VARCHAR, "Div3AirportID" VARCHAR, "Div3AirportSeqID" VARCHAR, "Div3WheelsOn" VARCHAR, "Div3TotalGTime" VARCHAR, "Div3LongestGTime" VARCHAR, "Div3WheelsOff" VARCHAR, "Div3TailNum" VARCHAR, "Div4Airport" VARCHAR, "Div4AirportID" VARCHAR, "Div4AirportSeqID" VARCHAR, "Div4WheelsOn" VARCHAR, "Div4TotalGTime" VARCHAR, "Div4LongestGTime" VARCHAR, "Div4WheelsOff" VARCHAR, "Div4TailNum" VARCHAR, "Div5Airport" VARCHAR, "Div5AirportID" VARCHAR, "Div5AirportSeqID" VARCHAR, "Div5WheelsOn" VARCHAR, "Div5TotalGTime" VARCHAR, "Div5LongestGTime" VARCHAR, "Div5WheelsOff" VARCHAR, "Div5TailNum" VARCHAR, "Unnamed: 109" VARCHAR))
SELECT
  TIME_FLOOR(TIME_PARSE("depaturetime"), 'PT1H') AS "__time",
  "Reporting_Airline",
  "Origin",
  "Dest",
  COUNT(*) AS "Events",
  MAX("Distance") AS "Distance_Max",
  MIN("Distance") AS "Distance_Min",
  DS_HLL("Tail_Number") AS "Tail_Number_HLL",
  DS_THETA("Tail_Number") AS "Tail_Number_THETA"
FROM "ext"
GROUP BY 1, 2, 3, 4
PARTITIONED BY DAY
'''

When doing this programmatically we need to be sure to include a context parameter that prompts Druid to store the true sketch value: [`finalizeAggregations`](https://druid.apache.org/docs/26.0.0/multi-stage-query/reference.html#context-parameters). Notice that, if you build an ingestion using the console, these settings are applied for you automatically.

The following cell adds the parameters and then executes the ingestion.

When the ingestion is finished, you will see the table definition. Monitor the ingestion task itself in the Druid Console.

In [ ]:
req = sql_client.sql_request(sql)
req.add_context("finalize", "false")
req.add_context("finalizeAggregations", "false")

sql_client.run_task(req)
sql_client.wait_until_ready('flights-counts')
display.table('flights-counts')

As noted before, use specific SQL functions when doing analytics directly on sketches. For `COUNT(DISTINCT)` these are:

* For HLL [`APPROX_COUNT_DISTINCT_DS_HLL`](https://druid.apache.org/docs/26.0.0/querying/sql-functions.html#approx_count_distinct_ds_hll), and
* for Theta [`APPROX_COUNT_DISTINCT_THETA`](https://druid.apache.org/docs/26.0.0/querying/sql-functions.html#approx_count_distinct_ds_theta).

Here's an example query showing our estimated results – notice the use of the `FILTER` clause to split results.

In [ ]:
sql='''
SELECT
   "Reporting_Airline",
   SUM("Distance_Max") AS "Miles_Flown",
   APPROX_COUNT_DISTINCT_DS_HLL("Tail_Number_HLL") FILTER (WHERE "Distance_Max" > 2000) AS "HLLApprox-over2k",
   APPROX_COUNT_DISTINCT_DS_THETA("Tail_Number_THETA") FILTER (WHERE "Distance_Max" < 2000) AS "ThetaApprox-under2k"
FROM "flights-counts"
GROUP BY 1
'''

display.sql(sql)

The next query generates three HLL sketches covering flights out of three cities in the United States over a three week period.

Sketch objects are mergable – this is why Druid is able to make use of them during massively-parallelised query execution. The next query takes advantage of this to estimate how many distinct `Tail_Number`s there were across all three sets.

You'll recognise the `APPROX_COUNT_DISTINCT_DS_HLL` function and the `DS_HLL` function, generating sketches for the `Tail_Number`s originating in each city. To that the query adds the `HLL_SKETCH_UNION` function, which does the merging of each independent set of result sets.

To turn it from a sketch into something readable, the query then uses the `HLL_SKETCH_ESTIMATE` function to give us a number instead of a sketch.

It's also grouping those calculations by weeks by using `TIME_FLOOR`.

In [ ]:
sql='''
SELECT
  TIME_FLOOR("__time",'P1W') AS "Week commencing",
  APPROX_COUNT_DISTINCT_DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='ATL') AS "From Atlanta",
  APPROX_COUNT_DISTINCT_DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='DFW') AS "From Dallas",
  APPROX_COUNT_DISTINCT_DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='SFO') AS "From San Francisco",
  HLL_SKETCH_ESTIMATE(
     HLL_SKETCH_UNION(
       DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='ATL'),
       DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='DFW'),
       DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='SFO')
      )
    ) AS "From any of the three",
  APPROX_COUNT_DISTINCT_DS_HLL("Tail_Number_HLL") AS "From any city"
FROM "flights-counts"
WHERE TIMESTAMP '2005-10-31' <= __time AND __time <= TIMESTAMP '2005-11-20'
GROUP BY 1
'''

display.sql(sql)

### Creating sketches during streaming ingestion

In streaming ingestion, rather than using `DS_HLL` or `DS_THETA` you include the Druid Native equivallent in the [`metricsSpec`](https://druid.apache.org/docs/26.0.0/ingestion/ingestion-spec.html#metricsspec) and - instead of `GROUP BY` - enable [`queryGranularity`](https://druid.apache.org/docs/latest/ingestion/ingestion-spec.html#granularityspec) and `rollup` to truncate the time stamp and pre-aggregate the rows.

The `INSERT` statement above is therefore equivallent to:

```json
    {
      "type": "HLLSketchBuild",
      "fieldName": "Tail_Number",
      "lgK": 12,
      "tgtHllType": "HLL_4"
    },
    {
      "type": "thetaSketch",
      "fieldName": "Tail_Number",
      "size": 16384
    }
```

Notice that here it's easy to see some internal parameters for sketch generation, like the `lgK` value for HLL. In SQL mode, these are exposed as supplementary parameters to the `DS_HLL` function. Be cautious of changing these values without researching the effects - not just in accuracy but also in terms of performance and segment size.

## Conclusion

* Approximation is the default execution model for `COUNT(DISTINCT)` queries
* You can turn it off with a query context parameter
* Accuracy is highly dependent on the distribution and cardinality of data across the database
* Druid can be pre-loaded with sketch objects that speed up approximation both in batch and streaming ingestion

## Learn more

* Watch [Employ Approximation](https://youtu.be/fSWwJs1gCvQ?list=PLDZysOZKycN7MZvNxQk_6RbwSJqjSrsNR) by Peter Marshall
* Read [Ingesting Data Sketches into Apache Druid](https://blog.hellmar-becker.de/2022/12/26/ingesting-data-sketches-into-apache-druid/) by Hellmar Becker
* Read more about the native "aggregator" functions for streaming ingestion
    * [ThetaSketch function](https://druid.apache.org/docs/26.0.0/development/extensions-core/datasketches-theta.html)
    * [HyperLogLog function](https://druid.apache.org/docs/26.0.0/development/extensions-core/datasketches-hll.html)